# 🎥 Face Movie Generator (MediaPipe — version robuste)

Notebook Colab prêt à l'emploi : détection MediaPipe (fallback Haar), musique optionnelle, et messages d'erreur clairs.

Utilisation : exécute toutes les cellules, upload tes images (1+), puis télécharge la vidéo.

In [ ]:
# Installer les dépendances
!pip install --quiet gradio opencv-python moviepy numpy mediapipe Pillow

In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
import mediapipe as mp
from PIL import Image
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
from tempfile import TemporaryDirectory
import traceback

# === Initialisation MediaPipe & Haar ===
mp_face_detection = mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_face(image_rgb, size=(640, 640)):
    """
    image_rgb: numpy array RGB (H, W, 3)
    Retourne : face RGB resized ou None
    """
    try:
        h, w, _ = image_rgb.shape
        # MediaPipe attend RGB uint8
        img_uint8 = image_rgb.astype(np.uint8)
        results = mp_face_detection.process(img_uint8)
        if results and results.detections:
            det = results.detections[0]
            bbox = det.location_data.relative_bounding_box
            x1 = max(int(bbox.xmin * w) - 20, 0)
            y1 = max(int(bbox.ymin * h) - 20, 0)
            x2 = min(int((bbox.xmin + bbox.width) * w) + 20, w)
            y2 = min(int((bbox.ymin + bbox.height) * h) + 20, h)
            face_img = image_rgb[y1:y2, x1:x2]
            face_img = cv2.resize(face_img, size)
            return face_img
        # Fallback Haar (convertir en BGR gris pour Haar)
        gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        if len(faces) > 0:
            x, y, w_box, h_box = max(faces, key=lambda f: f[2]*f[3])
            margin = 0.4
            x1 = max(int(x - w_box*margin), 0)
            y1 = max(int(y - h_box*margin), 0)
            x2 = min(int(x + w_box + w_box*margin), image_rgb.shape[1])
            y2 = min(int(y + h_box + h_box*margin), image_rgb.shape[0])
            face_img = image_rgb[y1:y2, x1:x2]
            face_img = cv2.resize(face_img, size)
            return face_img
    except Exception as e:
        print('Erreur detect_face:', e)
        traceback.print_exc()
    return None

def create_face_movie(files, music=None, duration_per_face=2.0, zoom=True, transition=1.0):
    """
    files: list of uploaded file objects from Gradio
    music: uploaded audio file object or None
    Retour: (status_message, path_to_video_or_None)
    """
    try:
        with TemporaryDirectory() as tempdir:
            clips = []
            processed_count = 0
            debug_msgs = []

            for idx, file_obj in enumerate(files):
                try:
                    # Gradio fournit un fichier avec .name ; on ouvre via PIL
                    pil_img = Image.open(file_obj.name).convert('RGB')
                    img = np.array(pil_img)
                except Exception as e:
                    msg = f'Impossible de lire {getattr(file_obj, "name", str(file_obj))}: {e}'
                    print(msg)
                    debug_msgs.append(msg)
                    continue

                face = detect_face(img)
                if face is None:
                    msg = f'Pas de visage détecté dans {os.path.basename(file_obj.name)}'
                    print(msg)
                    debug_msgs.append(msg)
                    continue

                # MoviePy attend images en RGB numpy
                clip = ImageClip(face).set_duration(duration_per_face)
                if zoom:
                    clip = clip.resize(lambda t: 1 + 0.03 * t)
                # Apply crossfadein when concatenating later
                clips.append(clip)
                processed_count += 1

            if processed_count == 0:
                return ("Aucun visage détecté dans les images fournies.\n" + "\n".join(debug_msgs), None)

            # Apply crossfade between clips
            # We will set crossfadein on each clip except the first when concatenating manually
            # Use concatenate_videoclips with method='compose' and then set crossfadein per clip before concatenation
            # For simplicity, use the crossfadein method on each clip (MoviePy will handle overlaps)
            clips_with_fx = []
            for i, c in enumerate(clips):
                if i == 0:
                    clips_with_fx.append(c)
                else:
                    clips_with_fx.append(c.crossfadein(transition))

            final = concatenate_videoclips(clips_with_fx, method='compose')

            # Audio optional
            if music is not None:
                try:
                    audio_clip = AudioFileClip(music.name)
                    audio_clip = audio_clip.set_duration(final.duration)
                    final = final.set_audio(audio_clip)
                except Exception as e:
                    # If audio fails, continue without audio but report
                    msg = f"Erreur audio (le fichier audio sera ignoré) : {e}"
                    print(msg)
                    debug_msgs.append(msg)

            out_path = os.path.join(tempdir, 'face_movie.mp4')
            # Sauvegarde (affiche les logs d'encoding dans la cellule)
            final.write_videofile(out_path, fps=24, codec='libx264', audio_codec='aac')

            status_msg = f'Vidéo générée : {processed_count} visage(s) utilisé(s).'
            if debug_msgs:
                status_msg += '\n' + '\n'.join(debug_msgs)
            return (status_msg, out_path)

    except Exception as e:
        tb = traceback.format_exc()
        print('Erreur create_face_movie:', e)
        print(tb)
        return (f'Erreur inattendue : {e}\nVoir logs pour plus de détails.', None)

# === Interface Gradio ===
iface = gr.Interface(
    fn=create_face_movie,
    inputs=[
        gr.File(label='Images (multiple)', file_types=['image'], file_count='multiple'),
        gr.File(label='Musique (optionnel)', file_types=['audio'])
    ],
    outputs=[
        gr.Textbox(label='Statut', lines=5),
        gr.File(label='Télécharger la vidéo')
    ],
    title='Face Movie (MediaPipe — robuste)',
    description='Upload images (1+). La musique est optionnelle. Colab + iPad friendly.'
)

iface.launch(share=True)
